In [1]:
import os

import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [2]:
results_path = "/results/"
start_date='2020-01-01'
end_date='2020-04-16'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
start_hour_day = 8
end_hour_day = 20

# Compute entropy of neighborhood visits

In [4]:
pings_geocoded = spark.read.parquet(os.path.join(results_path,source, 'processed',country,'pings_geocoded'))


In [5]:
pings_geocoded = pings_geocoded.where((pings_geocoded.time >= start_date) & (pings_geocoded.time <= end_date))
pings_geocoded = pings_geocoded.withColumn("date", F.date_format('time','yyyy-MM-dd'))

In [6]:
w = Window.partitionBy("cuebiq_id")
tmp = pings_geocoded.groupBy("date", "cuebiq_id").agg(F.count(admin_id).alias("ind_count")).withColumn("tot_count", F.sum("ind_count").over(w))
tmp = tmp.withColumn("p", F.col("ind_count")/F.col("tot_count"))
entropy_date_id=tmp.groupby("date", "cuebiq_id").agg((-F.sum(F.col("p") * F.log2(F.col("p")))).alias("entropy"))
entropy_date_id = entropy_date_id.join(tmp.select("date", "cuebiq_id", F.col("tot_count").alias("n_pings")), on = ["date", "cuebiq_id"])


tmp = pings_geocoded.filter((F.hour(F.col("time")) >= 8) & (F.hour(F.col("time")) <= 20)).groupBy("date", "cuebiq_id").agg(F.count(admin_id).alias("ind_count")).withColumn("tot_count", F.sum("ind_count").over(w))
tmp = tmp.withColumn("p", F.col("ind_count")/F.col("tot_count"))
entropy_daytime_id= tmp.groupby("date", "cuebiq_id").agg((-F.sum(F.col("p") * F.log2(F.col("p")))).alias("entropy"))
entropy_daytime_id = entropy_daytime_id.join(tmp.select("date", "cuebiq_id", F.col("tot_count").alias("n_pings")), on = ["date", "cuebiq_id"])

tmp = pings_geocoded.filter(((F.hour(F.col("time")) >= 20) & (F.hour(F.col("time")) <= 23)) |((F.hour(F.col("time")) >= 0) & (F.hour(F.col("time")) <= 8))).groupBy("date", "cuebiq_id").agg(F.count(admin_id).alias("ind_count")).withColumn("tot_count", F.sum("ind_count").over(w))
tmp = tmp.withColumn("p", F.col("ind_count")/F.col("tot_count"))
entropy_nighttime_id= tmp.groupby("date", "cuebiq_id").agg((-F.sum(F.col("p") * F.log2(F.col("p")))).alias("entropy"))
entropy_nighttime_id = entropy_nighttime_id.join(tmp.select("date", "cuebiq_id", F.col("tot_count").alias("n_pings")), on = ["date", "cuebiq_id"])
 

In [7]:
results_path = '/dbfs/results/'

In [8]:

entropy_date_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'entropy_date_id.csv'), index=False)
entropy_daytime_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'entropy_daytime_id.csv'), index = False)
entropy_nighttime_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'entropy_nighttime_id.csv'), index = False)

# Figures